### FXCM API
* https://fxcm.github.io/rest-api-docs

In [ ]:
import fxcmpy
fxcmpy.__version__

In [ ]:
api = fxcmpy.fxcmpy(config_file='/home/babel/Python/fxcm.cfg', server='real')

## Retrieving Tick Data

In [ ]:
import time
import numpy as np
import pandas as pd
import datetime as dt
from pylab import mpl, plt

In [ ]:
plt.style.use('seaborn')

In [ ]:
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [ ]:
#all available currency pairs

from fxcmpy import fxcmpy_tick_data_reader as tdr

In [ ]:
print(tdr.get_available_symbols())

In [ ]:
#following code retrieves one weeks worth of tick data for EURUSD

start = dt.datetime(2020, 3, 30)  
stop = dt.datetime(2020, 4, 3)    

In [ ]:
td = tdr('EURUSD', start, stop)   #retrieves data file, unpacks it and stores the raw data in DF

In [ ]:
td.get_raw_data().info()    #return DF object with raw data (index values are str objects)

In [ ]:
td.get_data().info()    #return DF object for which the index has been transformed to a DatetimeIndex

In [ ]:
td.get_data().head()

In [ ]:
sub = td.get_data(start='2020-04-03 12:00:00', end='2020-04-03 12:15:00')  #picks a subset of data set

In [ ]:
sub.head()

In [ ]:
sub['Mid'] = sub.mean(axis=1)  #calculate the mid prices from the bid and ask prices

In [ ]:
sub['SMA'] = sub['Mid'].rolling(1000).mean()  #derives SMA values over intervals of 10000 ticks

In [ ]:
sub[['Mid', 'SMA']].plot(figsize=(15,10), lw=0.75);
plt.title('Historical mid tick prices for EURUSD and SMA',loc='center');

## Retrieving Candles Data

In [ ]:
#available symbols for which candles data is provided:

from fxcmpy import fxcmpy_candles_data_reader as cdr

In [ ]:
print(cdr.get_available_symbols())

In [ ]:
start = dt.datetime(2020, 3, 30)  
stop = dt.datetime(2020, 4, 3)    

In [ ]:
period = 'H1'

In [ ]:
candles = cdr('EURUSD', start, stop, period)

In [ ]:
data = candles.get_data()

In [ ]:
data.info()

In [ ]:
data[data.columns[:4]].tail()

In [ ]:
data[data.columns[4:]].tail()

In [ ]:
data['MidClose'] = data[['BidClose', 'AskClose']].mean(axis=1) #calculate the mid close prices from bid and ask

In [ ]:
data['SMA10'] = data['MidClose'].rolling(10).mean()   #calculate two SMAs
data['SMA50'] = data['MidClose'].rolling(50).mean()   

In [ ]:
data[['MidClose', 'SMA10', 'SMA50']].plot(figsize=(15,10));

## Working with the API

In [ ]:
instruments = api.get_instruments()
instruments

In [ ]:
print(instruments)

In [ ]:
#retrieves the most recent m1 bar prices for EURUSD

candles = api.get_candles('EUR/USD', period='m1', number=250)

In [ ]:
candles['askclose'].plot(figsize=(15,10));

In [60]:
#1
def output(data, dataframe):
    print('%3d | %s | %s | %6.5f, %6.5f' 
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'), 
             data['Rates'][0], data['Rates'][1]))

Exception in thread Thread-257:
Traceback (most recent call last):
  File "/home/babel/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/babel/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/babel/anaconda3/lib/python3.7/site-packages/socketio/client.py", line 613, in _handle_eio_message
    self._handle_event(pkt.namespace, pkt.id, pkt.data)
  File "/home/babel/anaconda3/lib/python3.7/site-packages/socketio/client.py", line 497, in _handle_event
    r = self._trigger_event(data[0], namespace, *data[1:])
  File "/home/babel/anaconda3/lib/python3.7/site-packages/socketio/client.py", line 547, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "/home/babel/anaconda3/lib/python3.7/site-packages/fxcmpy/fxcmpy.py", line 2555, in __on_price_update__
    callbacks[func](data, self.prices[symbol])
  File "<ipython-input-57-9f2ea732995d>", line 5, in output

Exception in thread Thread-265:
Traceback (most recent call last):
  File "/home/babel/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/babel/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/babel/anaconda3/lib/python3.7/site-packages/socketio/client.py", line 613, in _handle_eio_message
    self._handle_event(pkt.namespace, pkt.id, pkt.data)
  File "/home/babel/anaconda3/lib/python3.7/site-packages/socketio/client.py", line 497, in _handle_event
    r = self._trigger_event(data[0], namespace, *data[1:])
  File "/home/babel/anaconda3/lib/python3.7/site-packages/socketio/client.py", line 547, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "/home/babel/anaconda3/lib/python3.7/site-packages/fxcmpy/fxcmpy.py", line 2555, in __on_price_update__
    callbacks[func](data, self.prices[symbol])
  File "<ipython-input-57-9f2ea732995d>", line 5, in output

In [ ]:
#2
api.subscribe_market_data('EUR/USD', (output,))